In [2]:
import pandas as pd
df = pd.read_csv("Data/final_jihwan.csv")

In [4]:
df.head()

,Unnamed: 0,시설아이디,weeks,조사일자,표본번호,관부직경,엽병장,엽수,엽장,엽폭,...,초장,최종화방차수,화방 꽃수(소화수),supplyEC,supplyPH,innerCO2,innerHum,innerTemp,innerSolar,CO2
0,0,B농가,4,20231006,1,12.39,139.0,5.0,79.0,70.0,...,255.0,0.0,0.0,153.33,983.83,152253.13,12967.8,2616.99,19739.88,145.143536
1,1,B농가,4,20231006,2,12.59,146.0,5.0,78.0,77.0,...,251.0,0.0,0.0,153.33,983.83,152253.13,12967.8,2616.99,19739.88,145.143536
2,2,B농가,4,20231006,3,13.91,100.0,6.0,73.0,68.0,...,209.0,0.0,0.0,153.33,983.83,152253.13,12967.8,2616.99,19739.88,145.143536
3,3,B농가,4,20231006,4,10.36,111.0,4.0,83.0,80.0,...,264.0,0.0,0.0,153.33,983.83,152253.13,12967.8,2616.99,19739.88,145.143536
4,4,B농가,4,20231006,5,9.48,203.0,3.0,94.0,94.0,...,307.0,0.0,0.0,153.33,983.83,152253.13,12967.8,2616.99,19739.88,145.143536


In [5]:
df.columns

Index(['Unnamed: 0', '시설아이디', 'weeks', '조사일자', '표본번호', '관부직경', '엽병장', '엽수',
       '엽장', '엽폭', '개화수', '초장', '최종화방차수', '화방 꽃수(소화수)', 'supplyEC', 'supplyPH',
       'innerCO2', 'innerHum', 'innerTemp', 'innerSolar', 'CO2'],
      dtype='object')

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import KNeighborsRegressor
import joblib

def MultiOutputRegressorFunc_KNN(df, saveFileName):
    """
    MultiOutputRegressor model for multiple target prediction using KNN
    
    Parameters:
    df (pd.DataFrame): Input dataframe containing all features and targets
    saveFileName (str): Name of the file to save the trained model
    
    Returns:
    MultiOutputRegressor: Trained model
    """
    
    # Define features and targets
    features = ['supplyEC', 'supplyPH', 'innerCO2', 'innerHum', 'innerTemp', 'innerSolar', 'CO2']
    targets = ['관부직경', '엽병장', '엽수', '엽장', '엽폭', '개화수', '초장', '최종화방차수', '화방 꽃수(소화수)']
    
    # Prepare the input and target data
    X = df[features]
    y = df[targets]
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Initialize KNN regressor
    knn_regressor = KNeighborsRegressor(n_neighbors=3)
    
    # Initialize and fit MultiOutputRegressor
    multi_output_regressor = MultiOutputRegressor(knn_regressor)
    multi_output_regressor.fit(X_train, y_train)
    
    # Evaluate the model
    score = multi_output_regressor.score(X_test, y_test)
    print(f'Model score: {score}')
    
    # Make predictions
    predictions = multi_output_regressor.predict(X_test)
    
    # Print some example predictions
    print("\nExample predictions:")
    print("실제값".ljust(15), *[f"{target[:7]:7}" for target in targets])
    print("예측값".ljust(15), *[f"{target[:7]:7}" for target in targets])
    print("-" * 100)
    for i in range(min(5, len(predictions))):  # Print first 5 predictions
        print("실제값".ljust(15), *[f"{val:7.2f}" for val in y_test.iloc[i]])
        print("예측값".ljust(15), *[f"{val:7.2f}" for val in predictions[i]])
        print("-" * 100)
    
    # # Save the model
    # filename = f'../Server/MLModels/{saveFileName}.joblib'
    # print(f"\n모델을 {filename}에 저장합니다.")
    # joblib.dump(multi_output_regressor, filename)
    
    return multi_output_regressor

# 사용 예시:
# df = pd.read_csv('your_data.csv')  # 데이터 로드
# model = MultiOutputRegressorFunc_KNN(df, 'multioutput_knn_model')

In [11]:
MultiOutputRegressorFunc_KNN(df=df,saveFileName="finaltest")

Model score: 0.5160619515913027

Example predictions:
실제값             관부직경    엽병장     엽수      엽장      엽폭      개화수     초장      최종화방차수  화방 꽃수(소
예측값             관부직경    엽병장     엽수      엽장      엽폭      개화수     초장      최종화방차수  화방 꽃수(소
----------------------------------------------------------------------------------------------------
실제값               20.93  106.00    6.00  104.00  103.00    0.00  249.00    0.00    0.00
예측값               21.11  141.33    6.00  104.00  100.00    0.00  261.67    0.67    0.00
----------------------------------------------------------------------------------------------------
실제값               31.08  167.00    8.00   69.00   63.00    7.00  365.00    2.00    2.00
예측값               28.48  190.00    6.33   90.00   78.33    2.00  299.67    2.00    0.67
----------------------------------------------------------------------------------------------------
실제값               27.95  173.00    7.00   72.00   62.00    0.00  282.00    1.00    0.00
예측값               25.06  13

MultiOutputRegressor(estimator=KNeighborsRegressor(n_neighbors=3))

In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
import joblib

def MultiOutputRegressorFunc_RF(df, saveFileName):
    """
    MultiOutputRegressor model for multiple target prediction using Random Forest
    
    Parameters:
    df (pd.DataFrame): Input dataframe containing all features and targets
    saveFileName (str): Name of the file to save the trained model
    
    Returns:
    MultiOutputRegressor: Trained model
    """
    
    # Define features and targets
    features = ['supplyEC', 'supplyPH', 'innerCO2', 'innerHum', 'innerTemp', 'innerSolar', 'CO2']
    targets = ['관부직경', '엽병장', '엽수', '엽장', '엽폭', '개화수', '초장', '최종화방차수', '화방 꽃수(소화수)']
    
    # Prepare the input and target data
    X = df[features]
    y = df[targets]
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Initialize Random Forest regressor
    rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
    
    # Initialize and fit MultiOutputRegressor
    multi_output_regressor = MultiOutputRegressor(rf_regressor)
    multi_output_regressor.fit(X_train, y_train)
    
    # Evaluate the model
    score = multi_output_regressor.score(X_test, y_test)
    print(f'Model score: {score}')
    
    # Make predictions
    predictions = multi_output_regressor.predict(X_test)
    
    # Print some example predictions
    print("\nExample predictions:")
    print("실제값".ljust(15), *[f"{target[:7]:7}" for target in targets])
    print("예측값".ljust(15), *[f"{target[:7]:7}" for target in targets])
    print("-" * 100)
    for i in range(min(5, len(predictions))):  # Print first 5 predictions
        print("실제값".ljust(15), *[f"{val:7.2f}" for val in y_test.iloc[i]])
        print("예측값".ljust(15), *[f"{val:7.2f}" for val in predictions[i]])
        print("-" * 100)
    
    # Calculate and print feature importances
    feature_importances = np.mean([tree.feature_importances_ for tree in multi_output_regressor.estimators_], axis=0)
    print("\nFeature Importances:")
    for feature, importance in zip(features, feature_importances):
        print(f"{feature}: {importance:.4f}")
    
    # # Save the model
    # filename = f'../Server/MLModels/{saveFileName}.joblib'
    # print(f"\n모델을 {filename}에 저장합니다.")
    # joblib.dump(multi_output_regressor, filename)
    
    return multi_output_regressor

# 사용 예시:
# df = pd.read_csv('your_data.csv')  # 데이터 로드
# model = MultiOutputRegressorFunc_RF(df, 'multioutput_rf_model')

In [24]:
MultiOutputRegressorFunc_RF(df=df,saveFileName="finaltest")

Model score: 0.5817622343538149

Example predictions:
실제값             관부직경    엽병장     엽수      엽장      엽폭      개화수     초장      최종화방차수  화방 꽃수(소
예측값             관부직경    엽병장     엽수      엽장      엽폭      개화수     초장      최종화방차수  화방 꽃수(소
----------------------------------------------------------------------------------------------------
실제값               20.93  106.00    6.00  104.00  103.00    0.00  249.00    0.00    0.00
예측값               21.97  139.87    5.76  104.88   99.77    0.00  265.31    0.76    0.00
----------------------------------------------------------------------------------------------------
실제값               31.08  167.00    8.00   69.00   63.00    7.00  365.00    2.00    2.00
예측값               25.99  170.64    6.22   78.94   69.63    5.13  306.97    2.00    1.61
----------------------------------------------------------------------------------------------------
실제값               27.95  173.00    7.00   72.00   62.00    0.00  282.00    1.00    0.00
예측값               24.13  15

MultiOutputRegressor(estimator=RandomForestRegressor(random_state=42))